In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
from torchvision import transforms
from torch.utils.data import Dataset

#plot
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import numpy as np 
import json

In [2]:
import sys
sys.path.append('/cephyr/users/olebal/Alvis/lora_ensembles/equivariant-posteriors')

In [3]:
import json

# Read the JSON data from the file
with open("/cephyr/users/olebal/Alvis/lora_ensembles/equivariant-posteriors/tokenizer_vocabulary.json", "r") as json_file:
    tokenizer_vocabulary = json.load(json_file)

with open("/cephyr/users/olebal/Alvis/lora_ensembles/equivariant-posteriors/detokenized_text_2.json", "r") as json_file:
    detokenized_text = json.load(json_file)

In [5]:
detokenized_text

{'0': '',
 '1': '',
 '2': '',
 '3': '',
 '4': '',
 '5': '',
 '6': '',
 '7': '',
 '8': '',
 '9': '',
 '10': '',
 '11': '',
 '12': '',
 '13': '',
 '14': '',
 '15': '',
 '16': '',
 '17': '',
 '18': '',
 '19': '',
 '20': '',
 '21': '',
 '22': '',
 '23': '',
 '24': '',
 '25': '',
 '26': '',
 '27': '',
 '28': '',
 '29': '',
 '30': '',
 '31': '',
 '32': '',
 '33': '',
 '34': '',
 '35': '',
 '36': '',
 '37': '',
 '38': '',
 '39': '',
 '40': '',
 '41': '',
 '42': '',
 '43': '',
 '44': '',
 '45': '',
 '46': '',
 '47': '',
 '48': '',
 '49': '',
 '50': '',
 '51': '',
 '52': '',
 '53': '',
 '54': '',
 '55': '',
 '56': '',
 '57': '',
 '58': '',
 '59': '',
 '60': '',
 '61': '',
 '62': '',
 '63': '',
 '64': '',
 '65': '',
 '66': '',
 '67': '',
 '68': '',
 '69': '',
 '70': '',
 '71': '',
 '72': '',
 '73': '',
 '74': '',
 '75': '',
 '76': '',
 '77': '',
 '78': '',
 '79': '',
 '80': '',
 '81': '',
 '82': '',
 '83': '',
 '84': '',
 '85': '',
 '86': '',
 '87': '',
 '88': '',
 '89': '',
 '90': '',
 '91': ''

In [6]:
import numpy as np
import dill as pickle

def read_res_from_dill(path):  
    with open(path, 'rb') as fp:
        res = pickle.load(fp)   
    return res

def calculate_mean_softmax_probs (softmax_probs_ensemble: torch.Tensor) -> torch.Tensor:

    mean_softmax_probs = torch.mean(softmax_probs_ensemble, dim=0)
    return mean_softmax_probs


def calculate_entropy_ens(
    softmax_probs_ensemble: torch.Tensor, 
    dim: int = -1, 
    eps: float = 1e-9
) -> torch.Tensor:

    softmax_probs = calculate_mean_softmax_probs(softmax_probs_ensemble)
    entropy = calculate_entropy(softmax_probs, dim, eps)

    return entropy

def calculate_entropy(
    softmax_probs: torch.Tensor, 
    dim: int = -1, 
    eps: float = 1e-9
) -> torch.Tensor:
    #softmax_probs_topk, indices = torch.topk(softmax_probs, k = 10, dim = dim)
    #coordinates = torch.unravel_index(indices, softmax_probs.shape)
    softmax_probs_topk = softmax_probs
    
    #softmax_probs_topk = softmax_probs[coordinates]
    entropy = -torch.sum(softmax_probs_topk * torch.log(softmax_probs_topk + eps), dim=dim)
    return entropy

def calculate_outclass_probs(
    softmax_probs_ensemble: torch.Tensor
) -> torch.Tensor:

    mean_softmax_probs = calculate_mean_softmax_probs(softmax_probs_ensemble)
    print(f"Outclass index: {mean_softmax_probs.shape[-1]}")
    outclass_mean_softmax_probs = mean_softmax_probs[:, -1]

    return outclass_mean_softmax_probs

def calculate_mutual_information_ens(
    softmax_probs_ensemble: torch.Tensor
) -> torch.Tensor:

    entropy_of_the_mean_ens_prob = calculate_entropy_ens(softmax_probs_ensemble)
    mean_ens_entropy = torch.mean(calculate_entropy(softmax_probs_ensemble), dim=0)
    
    mutual_information = entropy_of_the_mean_ens_prob - mean_ens_entropy

    return mutual_information

In [45]:
import os
path_dirs = []
path_files = []
names = []

epochs = [0, 1, 3, 6, 9]

path_dirs.append(f"/mimer/NOBACKUP/groups/snic2022-22-448/lora_ensembles/ens_llm_lora_eval_custom_OB_HL_paper/qa_set_3/train_custom_language_dataset/ens/EPOCH/dataset_custom_language_dataset/")
path_files.append("member_probs_and_targets_Mistral-7B-v0d1_lr_8_dr_0d0_rl2_0d0_rl2_1d0.dill")
names.append("Lora Ens (M=1) \n (custom dataset)")



In [46]:
entropy_res_dic = {}
mutual_info_res_dic = {}
softprobs_res_dic = {}
targets_res_dic = {}

for epoch in epochs:
    for index, path_file in enumerate(path_files):
        name = names[index]+"_"+str(epoch)
        accumulated_ens_probs = []
        for member_id in range(1):
            full_path_file = os.path.join(path_dirs[index].replace("EPOCH", str(epoch)), "member_"+str(member_id), path_file)
            res = read_res_from_dill(full_path_file)
            accumulated_ens_probs.append(res['softmax_probs'].unsqueeze(0))
        
        
        softprobs = torch.cat(accumulated_ens_probs)
        targets = res['targets']
        #print(softprobs.shape)
        
        
        if softprobs.shape[1]>0:


            mean_softprobs = calculate_mean_softmax_probs(softprobs)
            
            entropies = calculate_entropy_ens(softprobs)
            mutual_info = calculate_mutual_information_ens(softprobs)

            entropy_res_dic[name] = entropies
            mutual_info_res_dic[name] = mutual_info
            softprobs_res_dic[name] = softprobs
            targets_res_dic[name] = targets

In [57]:
key = 'Lora Ens (M=1) \n (custom dataset)_0'
softmax_probs = softprobs_res_dic[key][0]
predictions = torch.argmax(softmax_probs, dim=-1)
max_probs, _ = torch.max(softmax_probs, dim=-1)

targets = targets_res_dic[key]
entropy = entropy_res_dic[key]
wrong = 0
total = 0
correct = 0
mutual_info = mutual_info_res_dic[key]
for index in range(predictions.shape[0]):
    if tokenizer_vocabulary[str(targets[index].item())]=="Yes":
        total+=1
        print("------------------------------------------------------------------------------------")
        if tokenizer_vocabulary[str(predictions[index].item())] == "No":
            wrong+=1
        if tokenizer_vocabulary[str(predictions[index].item())] == "Yes":
            correct+=1
        print("##########################################")
        print("Prediction Text Token: ", tokenizer_vocabulary[str(predictions[index].item())])
        print("Prob Prediction Token: ", max_probs[index])
        print("Target Text Token: ", tokenizer_vocabulary[str(targets[index].item())])
        print("Prob Target Token: ", softmax_probs[index, targets[index].item()])
        print("Entropy: ", entropy[index].item())
        print("Mutual Info: ", mutual_info[index].item())

------------------------------------------------------------------------------------
##########################################
Prediction Text Token:  

Prob Prediction Token:  tensor(0.4465)
Target Text Token:  Yes
Prob Target Token:  tensor(0.0022)
Entropy:  3.178523540496826
Mutual Info:  0.0
------------------------------------------------------------------------------------
##########################################
Prediction Text Token:  Yes
Prob Prediction Token:  tensor(0.3077)
Target Text Token:  Yes
Prob Target Token:  tensor(0.3077)
Entropy:  3.477444648742676
Mutual Info:  0.0
------------------------------------------------------------------------------------
##########################################
Prediction Text Token:  

Prob Prediction Token:  tensor(0.2744)
Target Text Token:  Yes
Prob Target Token:  tensor(0.0516)
Entropy:  3.8958895206451416
Mutual Info:  0.0
------------------------------------------------------------------------------------
##################

In [58]:
wrong, correct, total

(0, 29, 62)

In [69]:
key = 'Lora Ens (M=1) \n (custom dataset)_6'
softmax_probs = softprobs_res_dic[key][0]
predictions = torch.argmax(softmax_probs, dim=-1)
max_probs, _ = torch.max(softmax_probs, dim=-1)
correct = 0

targets = targets_res_dic[key]
entropy = entropy_res_dic[key]
wrong = 0
total = 0
mutual_info = mutual_info_res_dic[key]
for index in range(predictions.shape[0]):
    if tokenizer_vocabulary[str(targets[index].item())]=="Yes":
        total+=1
        print("------------------------------------------------------------------------------------")
        if tokenizer_vocabulary[str(predictions[index].item())] == "No":
            wrong+=1
        if tokenizer_vocabulary[str(predictions[index].item())] == "Yes":
            correct+=1
        print("##########################################")
        print("Prediction Text Token: ", tokenizer_vocabulary[str(predictions[index].item())])
        print("Prob Prediction Token: ", max_probs[index])
        print("Target Text Token: ", tokenizer_vocabulary[str(targets[index].item())])
        print("Prob Target Token: ", softmax_probs[index, targets[index].item()])
        print("Entropy: ", entropy[index].item())
        print("Mutual Info: ", mutual_info[index].item())

------------------------------------------------------------------------------------
##########################################
Prediction Text Token:  We
Prob Prediction Token:  tensor(0.2164)
Target Text Token:  Yes
Prob Target Token:  tensor(0.0008)
Entropy:  4.251045227050781
Mutual Info:  0.0
------------------------------------------------------------------------------------
##########################################
Prediction Text Token:  Yes
Prob Prediction Token:  tensor(0.3568)
Target Text Token:  Yes
Prob Target Token:  tensor(0.3568)
Entropy:  3.2126073837280273
Mutual Info:  0.0
------------------------------------------------------------------------------------
##########################################
Prediction Text Token:  We
Prob Prediction Token:  tensor(0.0884)
Target Text Token:  Yes
Prob Target Token:  tensor(0.0118)
Entropy:  4.79659366607666
Mutual Info:  0.0
------------------------------------------------------------------------------------
#################

In [67]:
wrong, correct, total

(0, 29, 62)